In [80]:
import pandas as pd
import numpy as np
# Text preprocessing steps - remove numbers, captial letters and punctuation
import re
import string
import nltk

In [88]:
filereader = pd.read_csv('books.csv', iterator=False, chunksize=1000)
chunk = filereader.get_chunk()
data = chunk.copy()
data

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,25933450,RJOVP071AVAJO,0439873800,84656342,There Was an Old Lady Who Swallowed a Shell!,Books,5,0,0,N,Y,Five Stars,I love it and so does my students!,2015-08-31
1,US,1801372,R1ORGBETCDW3AI,1623953553,729938122,I Saw a Friend,Books,5,0,0,N,Y,Please buy inchI Saw a Friendinch! Your childr...,My wife and I ordered 2 books and gave them as...,2015-08-31
2,US,5782091,R7TNRFQAOUTX5,142151981X,678139048,Black Lagoon Vol. 6,Books,5,0,0,N,Y,Shipped fast.,Great book just like all the others in the ser...,2015-08-31
3,US,32715830,R2GANXKDIFZ6OI,014241543X,712432151,If I Stay,Books,5,0,0,N,N,Five Stars,So beautiful,2015-08-31
4,US,14005703,R2NYB6C3R8LVN6,1604600527,800572372,Stars 'N Strips Forever,Books,5,2,2,N,Y,Five Stars,Enjoyed the author's story and his quilts are ...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,US,21073382,R1GT81BJMRCJA4,1118254368,710946003,Professional Baking 6e with Professional Bakin...,Books,3,4,5,N,Y,Too clinical not recommended by my pastry ins...,I got this book as a textbook for my college's...,2015-08-31
996,US,44188391,R449Z4BWLQ4WH,1516928180,481202040,Hooked: A Stepbrother Romance,Books,5,0,0,N,N,Great book..,I had the privilege to read this book. It was ...,2015-08-31
997,US,42397861,R3M75MJPYM033V,0763657697,320496701,Willow,Books,1,1,2,N,N,Historical fiction or historical fantasy?,This book caught my attention. I love Victoria...,2015-08-31
998,US,52233305,R6VL1L0KDE5XZ,0871951762,630581469,Gus Grissom: The Lost Astronaut (INDIANA BIOGR...,Books,5,0,0,N,Y,Full of detail and a great read,Great piece of work by author Boomhower. Some ...,2015-08-31


## EDA

In [89]:
# Remove 3 star reviews
data = data[data.star_rating!=3]

# Set 4/5 star reviews to positive, the rest to negative
data['sentiment'] = np.where(data['star_rating'] >= 4, 'positive', 'negative')

# Include only the sentiment and reviews columns
data = data[['review_date', 'customer_id', 'product_title', 'review_headline', 'sentiment', 'review_body', 'helpful_votes', 'total_votes', 'verified_purchase']]
data.head()

/Users/samypalaniappan/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,review_date,customer_id,product_title,review_headline,sentiment,review_body,helpful_votes,total_votes,verified_purchase
0,2015-08-31,25933450,There Was an Old Lady Who Swallowed a Shell!,Five Stars,positive,I love it and so does my students!,0,0,Y
1,2015-08-31,1801372,I Saw a Friend,Please buy inchI Saw a Friendinch! Your childr...,positive,My wife and I ordered 2 books and gave them as...,0,0,Y
2,2015-08-31,5782091,Black Lagoon Vol. 6,Shipped fast.,positive,Great book just like all the others in the ser...,0,0,Y
3,2015-08-31,32715830,If I Stay,Five Stars,positive,So beautiful,0,0,N
4,2015-08-31,14005703,Stars 'N Strips Forever,Five Stars,positive,Enjoyed the author's story and his quilts are ...,2,2,Y


In [90]:
data.sentiment.value_counts(normalize=True)

positive    0.923574
negative    0.076426
Name: sentiment, dtype: float64

In [91]:
display(f'Total Number of rows', len(data))
unique_books = data.product_title.nunique()
unique_books

'Total Number of rows'

929

920

In [92]:
data.product_title.value_counts(ascending=False)

Serafina and the Black Cloak                                                        3
Plunder and Deceit: Big Government's Exploitation of Young People and the Future    3
Go Set a Watchman: A Novel                                                          3
Bill O'Reilly's Legends and Lies: The Real West                                     2
Adios  America                                                                      2
                                                                                   ..
Transformed                                                                         1
How Proust Can Change Your Life                                                     1
The All American Game                                                               1
Sistrsic92 (Meg)                                                                    1
Hush Now  Baby                                                                      1
Name: product_title, Length: 920, dtype: int64

## Preprocess the text

In [86]:
# Text preprocessing steps - remove numbers, captial letters and punctuation
import re
import string

symbols = lambda x: re.sub('\W+', ' ', x)
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

data['review_body'] = data.review_body.map(symbols).map(alphanumeric).map(punc_lower)
data.head()

TypeError: expected string or bytes-like object